In [22]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import warnings 
warnings.filterwarnings("ignore")

### Lidando com dados em branco

Substituindo dados em branco com um modelo de regressão, será usado o dataset de cerveja.

In [23]:
df = pd.read_csv('recipeData.csv', encoding= "ISO-8859-1")

# Contar as ocorrências de cada StyleID
contagem_classe = df['StyleID'].value_counts()

# Filtrar o DataFrame para manter apenas as linhas com mais de 1000 amostras
df_filtrado = df[df['StyleID'].isin(contagem_classe[contagem_classe > 1000].index)]

percen_faltantes = round((df_filtrado.isnull().sum() / df_filtrado.shape[0] )*100, 2)
percen_faltantes

BeerID            0.00
Name              0.01
URL               0.00
Style             0.00
StyleID           0.00
Size(L)           0.00
OG                0.00
FG                0.00
ABV               0.00
IBU               0.00
Color             0.00
BoilSize          0.00
BoilTime          0.00
BoilGravity       3.67
Efficiency        0.00
MashThickness    40.92
SugarScale        0.00
BrewMethod        0.00
PitchRate        55.19
PrimaryTemp      31.80
PrimingMethod    91.84
PrimingAmount    94.20
UserId           69.32
dtype: float64

In [24]:
# Váriaveis strings
sugar = pd.get_dummies(df_filtrado['SugarScale']).astype(int)
brew = pd.get_dummies(df_filtrado['BrewMethod']).astype(int)
concatenado = pd.concat([df_filtrado, sugar, brew], axis=1)

# Preenchendo dados faltantes pela mediana, performa melhor caso tenha outliers
concatenado['PrimaryTemp'].fillna(concatenado['PrimaryTemp'].median(), inplace= True ) 
concatenado['BoilGravity'].fillna(concatenado['BoilGravity'].median(), inplace= True ) 

df_missing = concatenado.drop(['BeerID', 'Name', 'URL', 'Style', 'PrimingAmount', 'PrimingMethod', 'UserId',
         'MashThickness', 'SugarScale', 'BrewMethod']
        , axis=1)

In [25]:
x_treino = df_missing[df_missing['PitchRate'].notnull()]
x_treino.drop('PitchRate', axis = 1, inplace=True)
y_treino = df_missing[df_missing['PitchRate'].notnull()]['PitchRate']

x_prever = df_missing[df_missing['PitchRate'].isnull()]
x_prever.drop('PitchRate', axis = 1, inplace=True)
y_prever = df_missing[df_missing['PitchRate'].isnull()]['PitchRate']

In [26]:
# Aplicando modelo
modelo = DecisionTreeRegressor()
modelo.fit(x_treino,y_treino)

# Predição dos valores missing
y_prever = modelo.predict(x_prever)

In [27]:
df_missing.PitchRate[df_missing.PitchRate.isnull()] = y_prever

In [28]:
percen_faltantes1 = round((df_missing.isnull().sum() / df_missing.shape[0] )*100, 2)
percen_faltantes1

StyleID             0.0
Size(L)             0.0
OG                  0.0
FG                  0.0
ABV                 0.0
IBU                 0.0
Color               0.0
BoilSize            0.0
BoilTime            0.0
BoilGravity         0.0
Efficiency          0.0
PitchRate           0.0
PrimaryTemp         0.0
Plato               0.0
Specific Gravity    0.0
All Grain           0.0
BIAB                0.0
Partial Mash        0.0
extract             0.0
dtype: float64